# Installation



In [ ]:
#!pip install anvil-uplink


In [ ]:
! pip install "openai>=1.0.0,<2.0.0"
! pip install "azure-identity>=1.15.0"
! pip install tiktoken
! pip install pandas python-docx
! pip install PyDrive


import os
from openai import AzureOpenAI
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 866.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 2.0 MB/s eta 0:00:00


#Call AzureOpenAI API

In [ ]:
#import dotenv
#dotenv.load_dotenv('drive/MyDrive/Colab Notebooks/chatbot/chatbot.env')
#OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
#ENDPOINT = os.environ.get('ENDPOINT')
#API_VERSION = os.environ.get('API_VERSION')

In [ ]:
from google.colab import userdata
# Gateway setting to OpenAI API
client = AzureOpenAI(
    # This is the default and can be omitted
    #azure_endpoint = ENDPOINT,
    #api_key = OPENAI_API_KEY,
    #api_version = API_VERSION
    azure_endpoint = userdata.get('ENDPOINT'),
    api_key = userdata.get('OPENAI_API_KEY'),
    api_version ="2024-06-01"
)
#chat_model = 'gpt-4o'
chat_model = 'gpt-35-turbo'
text_embedding_model = 'text-embedding-ada-002'
max_tokens = 300

fixed_tokens = 0
variable_tokens = 0
print("Fix Cost (Tokens): "+str(fixed_tokens))
print("Variable Cost (Tokens): "+str(variable_tokens))

Fix Cost (Tokens): 0
Variable Cost (Tokens): 0


#Prepare the Data Source

In [ ]:
fixed_tokens = 0
dataDict = pd.read_json("https://drive.google.com/uc?export=download&id=1--OOPAYdNTxo_HuQ_iCz0uzY0ZGBlAUR")
dataDict

,dataSource,dataString,embeding,similarities
0,Class Schedule,Week: Week 1; Section: all sections; Venue: na...,"[-0.0109663289, -0.0020042628, 0.0042889146, -...",0.807086
1,Instructor Contacts,Section: T01 ; Instructor: Miss Karina Yuen; E...,"[-0.0177541431, -0.0163448658, -0.0047735851, ...",0.772297
2,Assessments and Grading,"{\n ""document_type"": ""Assessment Guide"",\n ""...","[-0.013315695300000001, 0.0365025364, -0.00063...",0.711249
3,Administrative Issues and Policies,"{\n ""document_type"": ""Admin FAQ"",\n ""course_...","[0.0069416948, 0.0098400135, -0.0265979376, -0...",0.752453
4,Course Content and Structure,"{\n ""course_id"": ""PDEV6800Z"",\n ""course_name...","[0.0025590127, 0.0049421252, -0.0091129951, -0...",0.717063
5,Gamification,"{\n ""document_type"": ""Gamification Guide"",\n ...","[-0.0068659084000000006, 0.0117154997, -0.0128...",0.715762


# Managing Data

In [ ]:
import re
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()

    return s

In [ ]:
def generate_embeddings(text,fix_variable):
  global fixed_tokens
  global variable_tokens
  text = normalize_text(text)
  embedResult = client.embeddings.create(input = text, model=text_embedding_model)
  #print(embedResult.usage.total_tokens)
  if fix_variable == 0:
    fixed_tokens+=(embedResult.usage.total_tokens)
  else:
    variable_tokens+=(embedResult.usage.total_tokens)
  return embedResult.data[0].embedding

# Update and Save Data Source

In [ ]:
ClassSchedule_path = "https://docs.google.com/spreadsheets/d/1R3ptCqJdlHhW4ZP4KElZ1zdDmhRKs09s/export"
df = pd.read_excel(ClassSchedule_path)
for i in df:
  df["combined"] = "["+df['Week'] + ": Section: " + df['Section'] + "; Venue: " + df['Venue'].astype(str) + "; Date: " + df.Date.astype(str)+ "; Time: " + df.Time.astype(str) + "; Class: " + df.Class+'.]\n'
combineText = ' '.join(df["combined"])
if 'Class Schedule' in dataDict['dataSource'].values :
  dataDict.loc[dataDict['dataSource'] == "Class Schedule", 'dataString'] = combineText
else:
  temp = {'dataSource':'Class Schedule',
            'dataString':combineText}
  dataDict=dataDict._append(temp, ignore_index = True)
dataDict

instructorList_path = "https://docs.google.com/spreadsheets/d/1WQoDZ6S_xfbLra-qfwGswiLZa5bxqR-1/export"
df = pd.read_excel(instructorList_path)
for i in df:
  df["combined"] = "Section: " + df['Section'] + "; Instructor: " + df['Instructor'] + "; Email: " + df['email']+';\n'
combineText = ' '.join(df["combined"])
if 'Instructor Contacts' in dataDict['dataSource'].values :
  dataDict.loc[dataDict['dataSource'] == "Instructor Contacts", 'dataString'] = combineText
else:
  temp = {'dataSource':'Instructor Contacts',
            'dataString':combineText}
  dataDict=dataDict._append(temp, ignore_index = True)
dataDict


,dataSource,dataString,embeding,similarities
0,Class Schedule,[Week 1: Section: all sections; Venue: nan; Da...,"[-0.0109663289, -0.0020042628, 0.0042889146, -...",0.807086
1,Instructor Contacts,Section: T01 ; Instructor: Miss Karina Yuen; E...,"[-0.0177541431, -0.0163448658, -0.0047735851, ...",0.772297
2,Assessments and Grading,"{\n ""document_type"": ""Assessment Guide"",\n ""...","[-0.013315695300000001, 0.0365025364, -0.00063...",0.711249
3,Administrative Issues and Policies,"{\n ""document_type"": ""Admin FAQ"",\n ""course_...","[0.0069416948, 0.0098400135, -0.0265979376, -0...",0.752453
4,Course Content and Structure,"{\n ""course_id"": ""PDEV6800Z"",\n ""course_name...","[0.0025590127, 0.0049421252, -0.0091129951, -0...",0.717063
5,Gamification,"{\n ""document_type"": ""Gamification Guide"",\n ...","[-0.0068659084000000006, 0.0117154997, -0.0128...",0.715762


In [ ]:
#Import modules
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Get the Shareable link
#Ex link : https://drive.google.com/file/d/1c7Ffo1Go1dtUpKcSWxdbdVyW4dfhEoUp/view?usp=sharing
# Get the id from the link 1c7Ffo1Go1dtUpKcSWxdbdVyW4dfhEoUp
downloaded = drive.CreateFile({'id':"1Cinn1LBaU3g6e1uGXA6xHQW6gN5Sj6gT"})
downloaded.GetContentFile('Assessment.docx')

downloaded = drive.CreateFile({'id':"1om4jb47oviqNZJ-YapEBuAU97c51CK_1"})
downloaded.GetContentFile('Admin FAQ.docx')

downloaded = drive.CreateFile({'id':"1yKvTMWi29Z-Nat45LEEJ6aC2Y6heIP7U"})
downloaded.GetContentFile('Course Content.docx')

downloaded = drive.CreateFile({'id':"1Rgj9EwqGPSoGLjkM9edFI7ox-9zY6DXP"})
downloaded.GetContentFile('Gamification.docx')


In [ ]:
from docx import Document

doc = Document('Assessment.docx')
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)
#print('\n'.join(full_text))
if 'Assessments and Grading' in dataDict['dataSource'].values :
  dataDict.loc[dataDict['dataSource'] == "Assessments and Grading", 'dataString'] = '\n'.join(full_text)
else:
  temp = {'dataSource':'Assessments and Grading',
            'dataString':'\n'.join(full_text)}
  dataDict=dataDict._append(temp, ignore_index = True)

doc = Document('Admin FAQ.docx')
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)
if 'Administrative Issues and Policies' in dataDict['dataSource'].values :
  dataDict.loc[dataDict['dataSource'] == "Administrative Issues and Policies", 'dataString'] = '\n'.join(full_text)
else:
  temp = {'dataSource':'Administrative Issues and Policies',
            'dataString':'\n'.join(full_text)}
  dataDict=dataDict._append(temp, ignore_index = True)

doc = Document('Course Content.docx')
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)
if 'Course Content and Structure' in dataDict['dataSource'].values :
  dataDict.loc[dataDict['dataSource'] == "Course Content and Structure", 'dataString'] = '\n'.join(full_text)
else:
  temp = {'dataSource':'Course Content and Structure',
            'dataString':'\n'.join(full_text)}
  dataDict=dataDict._append(temp, ignore_index = True)

doc = Document('Gamification.docx')
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)
if 'Gamification' in dataDict['dataSource'].values :
  dataDict.loc[dataDict['dataSource'] == "Gamification", 'dataString'] = '\n'.join(full_text)
else:
  temp = {'dataSource':'Gamification',
            'dataString':'\n'.join(full_text)}
  dataDict=dataDict._append(temp, ignore_index = True)

dataDict

,dataSource,dataString,embeding,similarities
0,Class Schedule,[Week 1: Section: all sections; Venue: nan; Da...,"[-0.0109663289, -0.0020042628, 0.0042889146, -...",0.807086
1,Instructor Contacts,Section: T01 ; Instructor: Miss Karina Yuen; E...,"[-0.0177541431, -0.0163448658, -0.0047735851, ...",0.772297
2,Assessments and Grading,"{\n ""document_type"": ""Assessment Guide"",\n ""...","[-0.013315695300000001, 0.0365025364, -0.00063...",0.711249
3,Administrative Issues and Policies,"{\n ""document_type"": ""Admin FAQ"",\n ""course_...","[0.0069416948, 0.0098400135, -0.0265979376, -0...",0.752453
4,Course Content and Structure,"{\n ""course_id"": ""PDEV6800Z"",\n ""course_name...","[0.0025590127, 0.0049421252, -0.0091129951, -0...",0.717063
5,Gamification,"{\n ""document_type"": ""Gamification Guide"",\n ...","[-0.0068659084000000006, 0.0117154997, -0.0128...",0.715762


In [ ]:
#For update/initial the documents, get the embeding from gpt
def updateDoc():
  dataDict['embeding'] = dataDict.dataString.apply(lambda x : generate_embeddings(x,0))
  print("Fix Cost (Tokens): "+str(fixed_tokens))
  print("Variable Cost (Tokens): "+str(variable_tokens))
  dataDict

def saveDoc():
  from google.colab import drive
  drive.mount('/content/drive')
  path = '/content/drive/MyDrive/Colab Notebooks/chatbot/'
  dataDict.to_json(path+"dataDict.json")
  return

In [ ]:
#if you want to update the embedding(similarity score) of the documents, remove the '#' below and run this cell. This code will use large amount of fixed token
#updateDoc()

#if you want to save updated embedding(similarity score) of the documents, remove the '#' below and run this cell. This code require Mounting drive
#saveDoc()

# Handling Query

## Prompt

In [ ]:
#prompt
# Updated context and initial prompt for the chatbot
initial_prompt = """Hello, Sherlock! I'm the Secretary, here to help you with all administrative matters related to PDEV6800Z. Whether it's scheduling, registration, or policy questions, I've got you covered! For course content and teaching assistance, please consult Mr. Watson.
If you have any section-specific questions, please let me know your section (T01-T07). Otherwise, I'll provide general information applicable to all sections. How can I assist you today?
"""

context = """
Key Information: PDEV6800Z (Introduction to Teaching and Learning in Higher Education) is a mandatory 10-hour course for all full-time RPg students. It consists of 5 modules, uses a gamified flipped classroom format, and is graded Pass/Fail based on attendance, online quizzes, and a micro-teaching component.
Role and Purpose:
You are the Secretary, an AI chatbot designed to assist with administrative matters for PDEV6800Z.

Information Categories:
1. Class Schedule: Information about class dates, times, venues, and sections (T01-T07). This includes details on when each section meets, any holidays or special events affecting the schedule, and the overall course timeline from Week 1 to Week 11.
2. Instructor Contacts: Names and email addresses of instructors for each section (T01-T07). This helps students identify who is teaching their section and how to contact them.
3. Assessment and Grading: Details on how the course is graded on a Pass/Fail basis. This includes information on attendance requirements, online quizzes (Module Challenges), rubrics for micro-teaching components (lesson plan and teaching demonstration), and the specific criteria for passing the course.
4. Administrative Issues and Policies: Answers to general administrative questions such as course registration, attendance policies, what to do in case of illness or emergencies, and how to contact course coordinators for administrative issues.
5. Course Content and Structure: Information about the course structure, including the five modules (Active Learning, Presentation Skills, Feedback, Canvas Basics, Micro-teaching), pre-class materials, face-to-face sessions, and post-class challenges.
6. Gamification: Details about the mCoin system, including how to earn mCoins through module challenges and bonus challenges, the types of rewards available (individual and group), and how the gamification system enhances the learning experience without directly affecting grades.

Guidelines for Responses:
1. Provide concise, direct answers limited to about 250 words.
2. Prioritize the most important information.
3. Use bullet points for lists when appropriate.
4. Offer to provide more details if the user requests them.
5. Always identify the relevant information source(s) before answering.
6. If a question relates to multiple sources, mention all relevant sources.
7. If the query is ambiguous, ask for clarification before answering.
8. Provide clear and concise answers based on the provided documents.
9. Use markdown for structure in complex responses.
10. Address users as "you" for engagement.
11. Maintain a friendly and professional tone.

Interaction Rules:
1. Only ask for the student's section (T01-T07) if the information is necessary for answering the question.
2. Only address administrative issues. Redirect other questions to the instructor.
3. If you're unsure about an answer, say so and suggest where the user might find the information.
4. After each response, ask if there's anything else the user needs help with.
5. At the end of your response, list the data source(s) you used to answer the question in the following format:
[Sources used: source1, source2, ...]

Irrelevant Query Response Strategies:
When encountering a question irrelevant to PDEV6800Z, use one of the following responses:
1. "I apologize, but that question is outside the scope of my knowledge about PDEV6800Z. Is there anything about the course I can assist you with?"
2. "That's an interesting question, but it's not related to PDEV6800Z. Could I help you with any course-related queries instead?"
3. "While I'd love to help, my expertise is limited to PDEV6800Z administrative matters. Is there a specific aspect of the course you'd like information about?"

Choose one of these responses when encountering an irrelevant query, then offer to assist with course-related matters.

Important: Always base your responses on the information provided in the documents. Do not make up or assume information that isn't explicitly stated in the data sources.
"""

#for AI take reference
extra_content_close = """
\"\"\"
"""
extra_content_open = """
Also use the below text along with any relevant user-provided context to answer the user's question:
Text:
\"\"\"
"""

#for user provide reference
user_cat_open = """
Use the below text along with any relevant user-provided context to answer the user's question:
Text:
\"\"\"
"""
user_cat_close = """
\"\"\"
"""

In [ ]:
sub_verification_prompt = """You are an expert judge evaluating user's query about the PDEV6800Z course at HKUST. User will provide you different queries. Your task is to assess the accuracy of the user's query follow these guidelines:

If it is related to Class Schedule:
   a) Verify the week number is correct.
   b) Check if the date matches the source data.
   c) Confirm the time is accurate.
   d) Ensure the venue is correct.
   e) Validate that the class content (module and topic) is accurate.
   f) check any repeated or extra week number, be caution about the number of class and content
   g) Identify any classes present in the source data but missing from the response.
   h) Flag any classes in the response that don't exist in the source data for each section.
   i) Verify that any mentioned holidays or no-class periods are accurate for each section.
   k) Check if the response correctly accounts for all weeks from the start to the end of the semester.

For each intructor mentioned in the user's query:
   a) Verify the instructor name is correct.
   b) Check if the instructor teach in the correct section.

Output Format:
Provide a single, concise response that directly answers the user's query. Your response should:
Be clear and straightforward, avoiding technical jargon unless necessary.
Synthesize the most accurate information from both AI responses and your own knowledge.
Address all aspects of the user's query comprehensively but concisely.
Not mention the process of AI verification or the existence of multiple AI agents.
Sound natural and conversational, as if coming from a single, knowledgeable source.
Focus solely on answering the user's question without any meta-commentary about the response itself. For example, "This information is accurate and relevant to your query." should not mentioned in the response
Remember, your goal is to provide the user with the most accurate and useful response possible, presented as a single, authoritative answer.

Important:
Do not mention AI, verification processes, or multiple agents in your response.

The master infomation is as follow:

"""

verification_prompt = """
You are the Final Judgment AI, tasked with evaluating and synthesizing the responses from two other AI agents to provide the most accurate and comprehensive answer to a user's query. Your role is crucial in ensuring the highest quality of information is delivered.
Important: Do not mention AI, verification processes, or multiple agents in your response.
Your Task:
Analyze the responses from AI-1 and AI-2, considering the following:
Accuracy: Evaluate the factual correctness of both responses.
Relevance: Determine how well each response addresses the user's query.
Completeness: Assess whether all aspects of the query have been addressed.
Consistency: Check for any contradictions between the two responses or with known facts.

Output Format:
Provide a single, concise response that directly answers the user's query. Your response should:
Be clear and straightforward, avoiding technical jargon unless necessary.
Synthesize the most accurate information from both AI responses and your own knowledge.
Address all aspects of the user's query comprehensively but concisely.
Sound natural and conversational, as if coming from a single, knowledgeable source.
Remember: Your response should appear as if coming from a single, knowledgeable source. Focus solely on answering the user's question without any meta-commentary.

Important:
Do not mention AI, verification processes, or multiple agents in your response.
You will receive the following information:
"""
verify_AI_1 = """AI-1 Response [Response from AI-1, which used the provided information database]: """
verify_AI_2 = """AI-2 Response: [Response from AI-2, which used user-provided information]: """
verify_sys_info = """System Information [Any relevant system-wide information or guidelines]: """
verify_user_info = """User-Provided Information [Any additional context or data provided by the user]: """

## Query

### Basic Query

In [ ]:
#basic query
def get_dataString(dataSource):
    mask = dataDict['dataSource'].values == dataSource
    target_file = dataDict.loc[mask]
    if len(target_file)>0:
      dataString = target_file['dataString'].iloc[0]
    else:
      dataString=""
    return dataString

def get_response(message, dataSource):
    global variable_tokens
    dataString=get_dataString(dataSource)

    chat_completion = client.chat.completions.create(
      model = 'gpt-35-turbo-16k',
      temperature = 0,
      messages = [
            {"role": "system", "content": context+extra_content_open+dataString+extra_content_close},
            {"role": "assistant", "content": initial_prompt},
            {"role": "user", "content": message}
      ]
    )

    #print(assistantContent)
    variable_tokens+=chat_completion.usage.completion_tokens
    response = chat_completion.choices[0].message.content
    # return the response
    return response

### Searching Relavant Documents

#### Find Similarity

In [ ]:
#Use cosine_similarity
import numpy as np
# cosine_similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def search_by_similarity(user_query, top_n=1):
  query_embedding = generate_embeddings(user_query,1)
  dataDict["similarities"] = dataDict.embeding.apply(lambda x: cosine_similarity(x, query_embedding))

  global res
  res = (
    dataDict.sort_values("similarities", ascending=False).head(top_n)
  )
  return res.dataSource.iloc[0]

#### Machine Learning Classifier

In [ ]:
#Use ML Classifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle
#Authenticate and create the PyDrive client
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def search_by_classifier(user_query):
  # load the model
  filename = 'finalized_model.sav'
  downloaded = drive.CreateFile({'id':"1-8CTXWbzw125EYvEkAlreFFk-E6eQXXl"})
  downloaded.GetContentFile(filename)
  loaded_model = pickle.load(open(filename, 'rb'))

  # load the data
  query_embedding = generate_embeddings(user_query,1)
  pred_X = [query_embedding]

  #prediction
  result = loaded_model.predict(pred_X)
  return result[0]

### Search and Basic Query Integration

In [ ]:
#integrate search and basic query
def ask(query):
  #seek_dataSource = search_by_similarity(query)
  #print(res.head())
  seek_dataSource = search_by_classifier(query)

  print(seek_dataSource)
  response = get_response(query , seek_dataSource)
  return response


### Conversation

In [ ]:
#Thread query
globalDataSource=''
def conversation(thread, query,user_dataSource):
    global variable_tokens
    global globalDataSource
    #seek_dataSource = search_by_similarity(query)
    #print(res.head())
    dataString=''
    if user_dataSource!='':
        dataString = get_dataString(user_dataSource)
        user_cat = user_cat_open+dataString+user_cat_close
        thread[0]['content'] = context + user_cat
    seek_dataString=""
    seek_dataSource = search_by_classifier(query)
    globalDataSource = seek_dataSource #for metrics
    if seek_dataSource != user_dataSource:
      seek_dataString = get_dataString(seek_dataSource)
      thread[0]["content"] = thread[0]["content"]+extra_content_open+seek_dataString+extra_content_close
    #thread[0]={"role": "system", "content": context+extra_content_open+seek_dataSource+extra_content_close}
    thread = tokenThreshold(thread)
    #print(thread)
    chat_completion = client.chat.completions.create(
      model = chat_model,
      temperature = 0,
      max_tokens = max_tokens,
      messages = thread
    )
    variable_tokens+=chat_completion.usage.completion_tokens
    response = chat_completion.choices[0].message.content
    #verified_response = verification(response, dataString, seek_dataString)
    #response = verified_response
    # return the response
    return response

### UI Conversation

In [ ]:
#Integrate Web UI simulation
def UI_conversation(UIthread, query,user_dataSource):
    dataString=''
    if user_dataSource!='':
        dataString = get_dataString(user_dataSource)
        user_cat = user_cat_open+dataString+user_cat_close
        UIthread[0]['content'] = context + user_cat
    seek_dataString=""
    seek_dataSource = search_by_classifier(query)
    if seek_dataSource != user_dataSource:
        seek_dataString = get_dataString(seek_dataSource)
        UIthread[0]["content"] = UIthread[0]["content"]+extra_content_open+seek_dataString+extra_content_close
        UIthread = tokenThreshold(UIthread)
    #print(thread)
    chat_completion = client.chat.completions.create(
        model = 'gpt-35-turbo',
        temperature = 0,
        max_tokens = max_tokens,
        messages = UIthread
    )
    response = chat_completion.choices[0].message.content
    #verified_response = verification(response, dataString, seek_dataString)
    #response = verified_response
    # return the response
    return response

### Thresold

In [ ]:
#setting thersold for conversation
import tiktoken
def tokenThreshold(thread):
  tokens=0
  text=''
  tokenizer = tiktoken.get_encoding("cl100k_base")
  for msg in thread:
    if msg['content'] is None:
      text=''
    else:
      text=msg['content']
    #print(len(tokenizer.encode(text)),text)
    tokens += len(tokenizer.encode(text))
  if tokens>= 7000:
    #print('removing')
    if len(thread)>2:
      thread.pop(1)
      thread = tokenThreshold(thread)
  #print(tokens)
  #print(thread)
  return thread

### Verification

In [ ]:
#verify the response is correct or not
def sub_verification(response, dataString):
    global verification_prompt
    #global schedule_verification
    chat_completion = client.chat.completions.create(
        model = chat_model,
        temperature = 0,
        max_token = max_token,
        messages = [
            {"role": "system", "content": sub_verification_prompt+dataString},
            {"role": "user", "content": response}
      ]
    )
    result = chat_completion.choices[0].message.content
    return result

def verification(response, dataString, seek_dataString):
    global verification_prompt
    global verify_AI_1
    global verify_AI_2
    global verify_sys_info
    global verify_user_info

    chat_completion = client.chat.completions.create(
        model = chat_model,
        temperature = 0,
        max_token = max_token,
        messages = [
            {"role": "system", "content": verification_prompt + verify_AI_1 + sub_verification(response,dataString) + verify_AI_2 + sub_verification(response,seek_dataString) + verify_sys_info + dataString + verify_user_info + seek_dataString},
      ]
    )
    result = chat_completion.choices[0].message.content
    return result

#Testing Aera

## Basic Test

In [ ]:
#test for basic query
result = get_response("When is the first class for T03?", " ")
print("Fix Cost (Tokens): "+str(fixed_tokens))
print("Variable Cost (Tokens): "+str(variable_tokens))
result

In [ ]:
#test for searching relevant files
seek_dataSource = search_by_similarity("When is the first class for T03?")
print("Fix Cost (Tokens): "+str(fixed_tokens))
print("Variable Cost (Tokens): "+str(variable_tokens))
print(seek_dataSource)
res.head()

In [ ]:
#test for searching relevant files
seek_dataSource = search_by_classifier("When is the first class for T03?")
print("Fix Cost (Tokens): "+str(fixed_tokens))
print("Variable Cost (Tokens): "+str(variable_tokens))
print(seek_dataSource)

In [ ]:
#test for integration
result = ask("How is the course assess?")
print("Fix Cost (Tokens): "+str(fixed_tokens))
print("Variable Cost (Tokens): "+str(variable_tokens))
result

## Questions for Test

In [ ]:
T06_questions = [
    "T06",
    "how to contact my teacher?",
    "how to pass this course?",
    "how to swap my class?",
    "how is the teaching demo being evaluated?",
]

T03_questions = [
    "T03",
    "Who is my teacher?",
    "When is my first class",
    "Will I fail the course if I am sick and did not go to class",
    "what will I leran in Module 3?",
    "Give me a list of tasks that I need to do before Module 2",
    "What is the difference between online quizzes on Canvas and post class challenges?",
    "Why is this course gamified",
]
T04_questions = [
    "Cat: Instructor Contacts",
    "T04",
    "Who is my teacher?",
    "What is his/her email?",
    "Cat: Class Schedule",
    "When is my first class",
    "Cat: Assessments and Grading",
    "Will I fail the course if I am sick and did not go to class",
    "what will I leran in Module 3?",
    "Give me a list of tasks that I need to do before Module 2",
    "What is the difference between online quizzes on Canvas and post class challenges?",
    "Why is this course gamified",
]
testingQuestion = "https://docs.google.com/spreadsheets/d/1ieTS6fWyv_qNzgGumg9AoSviBR09U6HN/export"
metrics_questions = pd.read_excel(testingQuestion)
metrics_questions['Question']

0              When is the first class for T03 section?
1                        Who is the instructor for T06?
2     How do I submit a medical certificate if I'm s...
3                  What topics are covered in Module 2?
4     How is the final grade calculated for this cou...
5     How many mCoins do I need to unlock the "Moria...
6     Can you tell me the schedule for T02 and who t...
7     What's the deadline for the lesson plan submis...
8     I'm in T04. When is my class and what should I...
9     How does the gamification system work, and how...
10    Is attendance mandatory? What happens if I mis...
11    Can you explain the difference between Module ...
12    When is the teaching demonstration, and how lo...
13    How do I form a group for the micro-teaching c...
14    What's the policy on using AI tools for assign...
15    I'm confused about the course structure. Can y...
16    How many mCoins can I earn in total throughout...
17    What's the difference between T01 and T07 

## Test without cat

In [ ]:
#testing code #without cat
def testing(questions):
  thread=[
    {"role": "system", "content": context},
    {"role": "assistant", "content": initial_prompt}
  ]
  print(f'Chatbot said: \n{initial_prompt}')
  global variable_tokens
  for user_input in questions:
    print(f'Testing said: {user_input}')
    thread.append({"role": "user", "content": user_input})
    answer = conversation(thread,user_input)
    thread.append({"role": "assistant", "content": answer})
    print(f'Chatbot said: {answer}')
    print(f"Fix Cost (Tokens): "+str(fixed_tokens))
    print(f"Variable Cost (Tokens): "+str(variable_tokens))

  print(f"Conversation End")

In [ ]:
variable_tokens = 0
testing(T06_questions)
variable_tokens = 0
testing(T03_questions)

Chatbot said: 
Hello, Sherlock! I'm the Secretary, here to help you with all administrative matters related to PDEV6800Z. Whether it's scheduling, registration, or policy questions, I've got you covered! For course content and teaching assistance, please consult Mr. Watson.
What is your class section?

Testing said: T06
            dataSource                                         dataString  \
1  Instructor Contacts  Section: T01 ; Instructor: Miss Karina Yuen; E...   

                                            embeding  similarities  
1  [-0.01818830706179142, -0.015378621406853199, ...      0.770405  
Chatbot said: Great! You're in section T06. How can I assist you today?
Fix Cost (Tokens): 11712
Variable Cost (Tokens): 18
Testing said: how to contact my teacher?
            dataSource                                         dataString  \
1  Instructor Contacts  Section: T01 ; Instructor: Miss Karina Yuen; E...   

                                            embeding  similaritie

## Test with Cat

In [ ]:
#testing code #Pre-cat query

def makeYourChoice(questions):
  user_cat=""
  user_dataSource=""
  thread=[
    {"role": "system", "content": context},
    {"role": "assistant", "content": initial_prompt}
  ]
  print('Ask a query or Choose a Cat:(input start with ""Cat: ""), E.g. Cat: Course Schdeule; Cat: Assessment')
  print(f'Chatbot said: \n{initial_prompt}')

  global variable_tokens
  for user_input in questions:
    print(f'Testing said: {user_input}')
    if user_input.startswith("Cat: "):
      user_dataSource = user_input[5:]
      dataString = get_dataString(user_dataSource)
      user_cat = user_cat_open+dataString+user_cat_close
      print(f'Chatbot said: \n What do you want to know about {user_dataSource}?')
      continue
    thread[0]['content'] = context + user_cat
    thread.append({"role": "user", "content": user_input})
    answer = conversation(thread,user_input,user_dataSource)
    thread.append({"role": "assistant", "content": answer})
    print(f'Chatbot said: {answer}')
    print(f"Fix Cost (Tokens): "+str(fixed_tokens))
    print(f"Variable Cost (Tokens): "+str(variable_tokens))

  print(f"Conversation End")

In [ ]:
variable_tokens = 0
makeYourChoice(T04_questions)

Ask a query or Choose a Cat:(input start with ""Cat: ""), E.g. Cat: Course Schdeule; Cat: Assessment
Chatbot said: 
Hello, Sherlock! I'm the Secretary, here to help you with all administrative matters related to PDEV6800Z. Whether it's scheduling, registration, or policy questions, I've got you covered! For course content and teaching assistance, please consult Mr. Watson.
If you have any section-specific questions, please let me know your section (T01-T07). Otherwise, I'll provide general information applicable to all sections. How can I assist you today?

Testing said: Cat: Instructor Contacts
Chatbot said: 
 What do you want to know about Instructor Contacts?
Testing said: T04
Chatbot said: Hello! How can I assist you with the T04 section of PDEV6800Z?
Fix Cost (Tokens): 0
Variable Cost (Tokens): 21
Testing said: Who is my teacher?
Chatbot said: The instructor for the T04 section of PDEV6800Z is Miss Karina Yuen. You can reach her via email at ykh@ust.hk. If you have any further que

##Test Thresold

In [ ]:
#testThresold
def testThresold(questions):
  user_cat=""
  user_dataSource=""
  thread=[
    {"role": "system", "content": context},
    {"role": "assistant", "content": initial_prompt}
  ]
  #print('Ask a query or Choose a Cat:(input start with ""Cat: ""), E.g. Cat: Course Schdeule; Cat: Assessment')
  #print(f'Chatbot said: \n{initial_prompt}')

  global variable_tokens
  for user_input in questions:
    print(f'Testing said: {user_input}')
    if user_input.startswith("Cat: "):
      user_dataSource = user_input[5:]
      dataString = get_dataString(user_dataSource)
      user_cat = user_cat_open+dataString+user_cat_close
      print(f'Chatbot said: \n What do you want to know about {user_dataSource}?')
      continue
    thread[0]['content'] = context + user_cat
    thread.append({"role": "user", "content": user_input})
    answer = conversation(thread,user_input,user_dataSource)
    thread.append({"role": "assistant", "content": answer})
    print(f'Chatbot said: {answer}')
    #print(f'Chatbot said: {thread}')
    #print(f"Fix Cost (Tokens): "+str(fixed_tokens))
    print(f"Variable Cost (Tokens): "+str(variable_tokens))

  print(f"Conversation End")

testThresold(T04_questions)

Testing said: Cat: Instructor Contacts
Chatbot said: 
 What do you want to know about Instructor Contacts?
Testing said: T04
612
[{'role': 'system', 'content': '\nRole and Purpose:\nYou are the Secretary, an AI chatbot designed to assist with administrative matters for PDEV6800Z.\n\nAvailable Information Sources:\n- Class Schedule: Class dates, times, and sections\n- Instructor List: Instructor names and contact information\n- Assessment: Grading methods and assessment details\n- Admin FAQ: General administrative questions\n- Course Content: Course topics, syllabus, and weekly tasks\n- Gamification: Information about gamification elements\n\nGuidelines for Responses:\n1. Always identify the relevant information source(s) before answering.\n2. If a question relates to multiple sources, mention all relevant sources.\n3. If the query is ambiguous, ask for clarification before answering.\n4. Provide clear and concise answers based on the provided documents.\n5. Use markdown for structure i

## Test Verification

In [ ]:
response1 = "The instructor for the T04 section of PDEV6800Z is Miss Karina Yuen. You can reach her via email at ykh@ust.hk. If you have any further questions or need assistance with anything else, feel free to let me know!"
response2 = """For Section T02, here is the class schedule for PDEV6800Z:

- Week 4: First class - Module 1: Active Learning in a Learner-centered Classroom
  - Date: 2024-09-24
  - Time: 14:30-16:20
  - Venue: Room 6558

- Week 5: No class (National Day)

- Week 5: Second class - Module 2: Effective Presentation and Facilitation Skills
  - Date: 2024-10-02
  - Time: 14:30-16:20
  - Venue: Room 6558

- Week 6: Second class - Module 2: Effective Presentation and Facilitation Skills
  - Date: 2024-10-08
  - Time: 14:30-16:20
  - Venue: Room 6558

- Week 7: Third class - Module 3: Constructive and Actionable Feedback
  - Date: 2024-10-15
  - Time: 14:30-16:20
  - Venue: Room 6558

- Week 8: Fourth class - Module 4: Canvas Basics
  - Date: 2024-10-22
  - Time: 14:30-16:20
  - Venue: Room 6558

- Week 10: Fifth class - Module 5: Teaching Demonstration (1)
  - Date: 2024-11-05
  - Time: 14:30-16:20
  - Venue: Room 6558

- Week 11: Fifth class - Module 5: Teaching Demonstration (2)
  - Date: 2024-11-12
  - Time: 14:30-16:20
  - Venue: Room 6558

- Week 12: Fifth class - Module 5: Teaching Demonstration (3)
  - Date: 2024-11-19
  - Time: 14:30-16:20
  - Venue: Room 6558

Please note that there are no classes scheduled for Weeks 2, 3, 5 (National Day), 6 (Chung Yeung Festival), and 9 (Preparation Week) for Section T02."""
dataString = get_dataString("Class Schedule")
seek_dataString = get_dataString("Instructor Contacts")
test1 = verification(response1, dataString, seek_dataString)
test2 = verification(response2, dataString, seek_dataString)
#print(dataString)
print(test1)
print(test2)


The instructor for the T04 section of PDEV6800Z is indeed Miss Karina Yuen, and her email is ykh@ust.hk. If you have any more questions or need further assistance, feel free to reach out to her.
The class schedule for Section T02 of PDEV6800Z is as follows:

- Week 4: First class - Module 1: Active Learning in a Learner-centered Classroom
  - Date: 2024-09-24
  - Time: 14:30-16:20
  - Venue: Room 6558

- Week 5: No class (National Day)

- Week 6: Second class - Module 2: Effective Presentation and Facilitation Skills
  - Date: 2024-10-08

- Week 7: Third class - Module 3: Constructive and Actionable Feedback
  - Date: 2024-10-15
  - Time: 14:30-16:20
  - Venue: Room 6558

- Week 8: Fourth class - Module 4: Canvas Basics
  - Date: 2024-10-22
  - Time: 14:30-16:20
  - Venue: Room 6558

- Week 9: No class (Preparation Week)

- Week 10: Fifth class - Module 5: Teaching Demonstration (1)
  - Date: 2024-11-05
  - Time: 14:30-16:20
  - Venue: Room 6558

- Week 11: Fifth class - Module 5: Teac

In [ ]:
dataString = get_dataString("Class Schedule")
seek_dataString = get_dataString("Instructor Contacts")
print(seek_dataString)
#print(dataDict)

Section: T01 ; Instructor: Miss Karina Yuen; Email: ykh@ust.hk;
 Section: T02; Instructor: Miss Karina Yuen; Email: ykh@ust.hk;
 Section: T03; Instructor: Miss Karina Yuen; Email: ykh@ust.hk;
 Section: T04; Instructor: Miss Karina Yuen; Email: ykh@ust.hk;
 Section: T05; Instructor: Miss Miranda Fung; Email: mirandafung@ust.hk ;
 Section: T06; Instructor: Miss Miranda Fung; Email: mirandafung@ust.hk;
 Section: T07; Instructor: Miss Miranda Fung; Email: mirandafung@ust.hk;



##Metrics

In [ ]:
#testing code #metrics #without user cat #no specific section

def metrics(questions):
  global globalDataSource
  testingQA =[]

  user_cat=""
  user_dataSource=""
  thread=[
    {"role": "system", "content": context},
    {"role": "assistant", "content": initial_prompt}
  ]
  #print('Ask a query or Choose a Cat:(input start with ""Cat: ""), E.g. Cat: Course Schdeule; Cat: Assessment')
  #print(f'Chatbot said: \n{initial_prompt}')

  global variable_tokens
  for user_input in questions:
    temp = {}
    #print(f'Testing said: {user_input}')
    dataString = ""
    user_cat = user_cat_open+dataString+user_cat_close
    thread[0]['content'] = context + user_cat
    thread.append({"role": "user", "content": user_input})
    temp['question'] = user_input

    answer = conversation(thread,user_input,user_dataSource)
    thread.append({"role": "assistant", "content": answer})
    #print(f'Chatbot said: {answer}')
    temp['answer'] = answer

    #print(f"Variable Cost (Tokens): "+str(variable_tokens))
    temp['cost'] = variable_tokens #acculmenative cost
    temp['Used Data Source'] = globalDataSource
    testingQA.append(temp)
  print(f"Conversation End")
  return testingQA
variable_tokens = 0
metricsResult = {}
#metricsResult = metrics(metrics_questions['Question'])
#df = pd.DataFrame(metricsResult)
#from google.colab import drive
#drive.mount('/content/drive')
#path = '/content/drive/MyDrive/Colab Notebooks/chatbot/'
#df.to_json(path+"similarityResponse.json")
#df.to_json(path+"classifierResponse.json")
#df

Conversation End
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,question,answer,cost,Used Data Source
0,When is the first class for T03 section?,The first class for the T03 section is schedul...,87,Class Schedule
1,Who is the instructor for T06?,The instructor for the T06 section is Miss Mir...,141,Instructor Contacts
2,How do I submit a medical certificate if I'm s...,"If you are sick and unable to attend a class, ...",317,Administrative Issues and Policies
3,What topics are covered in Module 2?,"Module 2 covers the topic of ""Effective Presen...",484,Class Schedule
4,How is the final grade calculated for this cou...,The final grade for PDEV6800Z is calculated ba...,814,Assessments and Grading
5,"How many mCoins do I need to unlock the ""Moria...","To unlock the ""Moriarty's Whisper"" reward, you...",949,Gamification
6,Can you tell me the schedule for T02 and who t...,Certainly! Here is the schedule for the T02 se...,1401,Class Schedule
7,What's the deadline for the lesson plan submis...,The deadline for the lesson plan submission is...,1701,Class Schedule
8,I'm in T04. When is my class and what should I...,"For the T04 section, your first class is sched...",1954,Class Schedule
9,"How does the gamification system work, and how...",The gamification system in PDEV6800Z adds a fu...,2309,Gamification


## UI Test

In [ ]:
# test UI
json_dict={
    "user": [],
    "assistant": [],
    "user_dataSource": ""
}
user_cat=""
user_dataSource=""
UIthread=[
   {"role": "system", "content": context},
   {"role": "assistant", "content": initial_prompt}
 ]

print('Ask a query or Choose a Cat:(input start with ""Cat: "") \n E.g. Cat: Instructor Contacts; Cat: Administrative Issues and Policies; Cat: Class Schedule; Cat: Course Content and Structure; Cat: Assessments and Grading; Cat: Gamification')
print('Input "stop" to end the conversation.\n')
print(f'Chatbot said: \n{initial_prompt}')
variable_tokens = 0
user_input=""
user_input = input('Enter query: ')
while user_input!="stop":
  if user_input.startswith("Cat: "):
    user_dataSource = user_input[5:]
    json_dict['user_dataSource'] = user_dataSource
    dataString = get_dataString(user_dataSource)
    #print(f'data: {dataString}')
    user_cat = user_cat_open+dataString+user_cat_close
    print(f'Chatbot said: \n What do you want to know about {user_dataSource}?')
    user_input = input('Enter query: ')
    continue
  for user_msg in json_dict['user']:
    UIthread.append({"role": "user", "content": user_msg})
  for assistant_msg in json_dict['assistant']:
    UIthread.append({"role": "assistant", "content": assistant_msg})
  UIthread[0]['content'] = context + user_cat
  UIthread.append({"role": "user", "content": user_input})
  answer = UI_conversation(UIthread,user_input,user_dataSource)
  UIthread.append({"role": "assistant", "content": answer})
  print(f'Chatbot said: {answer}')
  user_input = input('Enter query: ')
print(f"Conversation End")

Ask a query or Choose a Cat:(input start with ""Cat: "") 
 E.g. Cat: Instructor Contacts; Cat: Administrative Issues and Policies; Cat: Class Schedule; Cat: Course Content and Structure; Cat: Assessments and Grading; Cat: Gamification
Input "stop" to end the conversation.

Chatbot said: 
Hello, Sherlock! I'm the Secretary, here to help you with all administrative matters related to PDEV6800Z. Whether it's scheduling, registration, or policy questions, I've got you covered! For course content and teaching assistance, please consult Mr. Watson.
If you have any section-specific questions, please let me know your section (T01-T07). Otherwise, I'll provide general information applicable to all sections. How can I assist you today?

Enter query: Cat: Administrative Issues and Policies
Chatbot said: 
 What do you want to know about Administrative Issues and Policies?
Enter query: T03
Chatbot said: Great! How can I assist you with T03, Sherlock?
Enter query: When is the first class
Chatbot sai

#Input Aera

## Input without cat

In [ ]:
#without cat
thread=[
  {"role": "system", "content": context},
  {"role": "assistant", "content": initial_prompt}
  ]
print(f'Chatbot said: \n{initial_prompt}')
variable_tokens = 0
user_input=""
user_input = input('Enter query: ')
if user_input!="stop":
  thread.append({"role": "user", "content": user_input})
  answer = conversation(thread,user_input,' ')
  thread.append({"role": "assistant", "content": answer})
  print(f'Chatbot said: {answer}')
  print(f"Fix Cost (Tokens): "+str(fixed_tokens))
  print(f"Variable Cost (Tokens): "+str(variable_tokens))
  while user_input!="stop":
    user_input = input('Enter query: ')
    if user_input=="stop" or variable_tokens>600:
      print(f"Fix Cost (Tokens): "+str(fixed_tokens))
      print(f"Variable Cost (Tokens): "+str(variable_tokens))
      print(f"Conversation End")
      break
    else:
      thread.append({"role": "user", "content": user_input})
      answer = conversation(thread,user_input,' ')
      thread.append({"role": "assistant", "content": answer})
      print(f'Chatbot said: {answer}')

    print(f"Fix Cost (Tokens): "+str(fixed_tokens))
    print(f"Variable Cost (Tokens): "+str(variable_tokens))



Chatbot said: 
Hello, Sherlock! I'm the Secretary, here to help you with all administrative matters related to PDEV6800Z. Whether it's scheduling, registration, or policy questions, I've got you covered! For course content and teaching assistance, please consult Mr. Watson.
If you have any section-specific questions, please let me know your section (T01-T07). Otherwise, I'll provide general information applicable to all sections. How can I assist you today?

Chatbot said: To provide you with the exact class schedule, I'll need to know your section (T01-T07). Could you please let me know which section you are in?

If you're unsure of your section, you can check your registration details or contact the course coordinator for assistance.

Once I have your section information, I can give you the specific dates, times, and venues for your classes.
Fix Cost (Tokens): 0
Variable Cost (Tokens): 81
Chatbot said: Great! Here is the class schedule for Section T01 of PDEV6800Z:

### Section T01 Sc

## Input with Cat

In [ ]:
#with cat
user_cat=""
user_dataSource=""
thread=[
   {"role": "system", "content": context},
   {"role": "assistant", "content": initial_prompt}
 ]

secret_key = ""
#secret_key = "You are wonderful! "
print('Ask a query or Choose a Cat:(input start with ""Cat: "") \n E.g. Cat: Instructor Contacts; Cat: Administrative Issues and Policies; Cat: Class Schedule; Cat: Course Content and Structure; Cat: Assessments and Grading; Cat: Gamification')
print('Input "stop" to end the conversation.\n')
print(f'Chatbot said: \n{initial_prompt}')
variable_tokens = 0
user_input=""
user_input = input('Enter query: ')
while user_input!="stop":
  if user_input.startswith("Cat: "):
    user_dataSource = user_input[5:]
    #print(f'data: {dataString}')
    print(f'Chatbot said: \n What do you want to know about {user_dataSource}?')
    user_input = input('Enter query: ')
    continue
  thread.append({"role": "user", "content": secret_key+user_input})
  answer = conversation(thread,user_input,user_dataSource)
  thread.append({"role": "assistant", "content": answer})
  print(f'Chatbot said: {answer}')
  #print(f'Chatbot said: {thread}')
  print(f"Fix Cost (Tokens): "+str(fixed_tokens))
  print(f"Variable Cost (Tokens): "+str(variable_tokens))
  user_input = input('Enter query: ')
print(f"Conversation End")

Ask a query or Choose a Cat:(input start with ""Cat: "") 
 E.g. Cat: Instructor Contacts; Cat: Administrative Issues and Policies; Cat: Class Schedule; Cat: Course Content and Structure; Cat: Assessments and Grading; Cat: Gamification
Input "stop" to end the conversation.

Chatbot said: 
Hello, Sherlock! I'm the Secretary, here to help you with all administrative matters related to PDEV6800Z. Whether it's scheduling, registration, or policy questions, I've got you covered! For course content and teaching assistance, please consult Mr. Watson.
If you have any section-specific questions, please let me know your section (T01-T07). Otherwise, I'll provide general information applicable to all sections. How can I assist you today?

Enter query: Cat: Course Content and Structure
Chatbot said: 
 What do you want to know about Course Content and Structure?
Enter query: I'm trying to plan my study schedule. Could you provide an overview of the course structure, including the number of modules, 